In [1]:
import arcpy
import requests
import os
#import psycopg2
import random
from pathlib import Path
import zipfile
import io

In [2]:
# Set workspace
os.chdir(r'C:\Users\eriks\OneDrive\Documents\ArcGIS\Projects\Lab4_arc2')
wksp = os.getcwd()

## 1. Data preparation

In [6]:
# Retrieve temperature data from PostGIS database
arcpy.management.MakeQueryLayer(
    input_database=os.path.join(wksp, "34.27.219.64.sde"),
    out_layer_name="temp_stations",
    query="SELECT id, min_tmpf, geom FROM stations WHERE date = '2023-03'",
    oid_fields="id",
    shape_type="POINT",
    srid="4326",
    spatial_reference='GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]];-400 -400 1000000000;0 1;0 1;8.98315284119521E-09;2;2;IsHighPrecision',
    spatial_properties="DO_NOT_DEFINE_SPATIAL_PROPERTIES",
    m_values="DO_NOT_INCLUDE_M_VALUES",
    z_values="DO_NOT_INCLUDE_Z_VALUES",
    extent='-98.0690216979786 43.2052294998382 -88.6618510633838 49.6779752981444 GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]'
)

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000837: The workspace is not the correct workspace type.
Failed to execute (MakeQueryLayer).


In [4]:
# Create a copy of the temperature as a shapefile in the workspace
arcpy.management.CopyFeatures(
    in_features="temp_stations",
    out_feature_class=os.path.join(wksp, "temp_stations.shp"),
    config_keyword="",
    spatial_grid_1=None,
    spatial_grid_2=None,
    spatial_grid_3=None
)

<Result 'E:\\ArcGIS_2\\Lab3\\temp_stations.shp'>

In [32]:
# Set the path to input temperature, and output training and validation shapefiles
input_shapefile = "stations.shp"
training_shapefile = os.path.join(wksp, "training_shapefile.shp")
validation_shapefile = os.path.join(wksp, "validation_shapefile.shp")

# Set the percentage of features to use for training
training_percent = 70

# Create a list of ObjectIDs for all features in the input shapefile
all_ids = [row[0] for row in arcpy.da.SearchCursor(input_shapefile, ["OID@"])]

# Calculate the number of features to use for training
num_training = int((len(all_ids) * training_percent) / 100)

# Randomly select the ObjectIDs for the training features
training_ids = random.sample(all_ids, num_training)

# Create separate lists of ObjectIDs for the validation and training features
validation_ids = [id for id in all_ids if id not in training_ids]

In [33]:
# Use the selected ObjectIDs to create new shapefiles for 
# i) training 
training = arcpy.management.SelectLayerByAttribute(input_shapefile, "NEW_SELECTION", "FID IN {}".format(tuple(training_ids)))
arcpy.management.CopyFeatures(training, training_shapefile)

# ii) validation 
validation = arcpy.management.SelectLayerByAttribute(input_shapefile, "NEW_SELECTION", "FID IN {}".format(tuple(validation_ids)))
arcpy.management.CopyFeatures(validation, validation_shapefile)

<Result 'C:\\Users\\eriks\\OneDrive\\Documents\\ArcGIS\\Projects\\Lab4_arc2\\validation_shapefile.shp'>

## 2. Interpolation

In [34]:
# Interpolate the temperature using 3 methods: IDW, Kriging, and GPI

arcpy.ddd.Idw(
    in_point_features="training_shapefile.shp",
    z_field="min_tmpf",
    out_raster=os.path.join(wksp, "IDW.tif"),
    cell_size=0.1,
    power=2,
    search_radius="VARIABLE 12",
    in_barrier_polyline_features=None
)

arcpy.ddd.Kriging(
    in_point_features="training_shapefile.shp",
    z_field="min_tmpf",
    out_surface_raster=os.path.join(wksp, "Kriging.tif"),
    semiVariogram_props="Spherical 0.021245 # # #",
    cell_size=0.1,
    search_radius="VARIABLE 12",
    out_variance_prediction_raster=None
)

arcpy.ga.GlobalPolynomialInterpolation(
    in_features="training_shapefile.shp",
    z_field="min_tmpf",
    out_ga_layer=None,
    out_raster=os.path.join(wksp, "GPI.tif"),
    cell_size=0.1,
    power=1,
    weight_field=None
)

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000732: Input features: Dataset training_shapefile.shp does not exist or is not supported
Failed to execute (GlobalPolynomialInterpolation).


In [20]:
arcpy.ddd.Kriging(
    in_point_features="training_shapefile.shp",
    z_field="max_tmpf",
    out_surface_raster=os.path.join(wksp, "Kriging_max.tif"),
    semiVariogram_props="Spherical 0.021245 # # #",
    cell_size=0.1,
    search_radius="VARIABLE 12",
    out_variance_prediction_raster=None
)

<Result 'C:\\Users\\eriks\\OneDrive\\Documents\\ArcGIS\\Projects\\Lab4_arc2\\Kriging_max.tif'>

### Append Interpolated Temperature Values to City Data

In [3]:
# Retrieve MN territories Data
territories_output = requests.post(r'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dot/bdry_mn_city_township_unorg/shp_bdry_mn_city_township_unorg.zip')
zipfile.ZipFile(io.BytesIO(territories_output.content)).extractall(wksp)

In [4]:
# Extract only the cities
arcpy.analysis.Select(
    in_features="city_township_unorg.shp",
    out_feature_class="cities.shp",
    where_clause="CTU_CLASS = 'CITY'"
)

<Result 'C:\\Users\\eriks\\OneDrive\\Documents\\ArcGIS\\Projects\\Arc2_Lab2\\cities.shp'>

In [5]:
# Set the input and output files
polygon_fc = "cities.shp"
centroid_fc = "centroids"
field_name = "min_temp"

# Check if the output feature class already exists, and if so, delete it
if arcpy.Exists(centroid_fc):
    arcpy.Delete_management(centroid_fc)

# Create the output feature class and add the shape field
arcpy.CreateFeatureclass_management(arcpy.env.workspace, centroid_fc, "POINT", spatial_reference=4326)

# Add all fields from the input polygon feature class to the output feature class
field_info = arcpy.ListFields(polygon_fc)
field_names = [field.name for field in field_info if not field.required]
for field in field_info:
    if not field.required:
        arcpy.AddField_management(centroid_fc, field.name, field.type, field.precision, field.scale, field.length, field.aliasName)

# Loop through the polygon features and calculate the centroids
with arcpy.da.SearchCursor(polygon_fc, ["SHAPE@", *field_names]) as in_cursor:
    with arcpy.da.InsertCursor(centroid_fc, ["SHAPE@", *field_names]) as out_cursor:
        for row in in_cursor:
            polygon = row[0]
            centroid = polygon.centroid
            out_cursor.insertRow((centroid,) + row[1:])

In [8]:
arcpy.sa.ExtractValuesToPoints(
    in_point_features="centroids.shp",
    in_raster="Kriging.tif",
    out_point_features=r"City_Min",
    interpolate_values="NONE",
    add_attributes="VALUE_ONLY"
)

<geoprocessing server result object object at 0x00000297E75B7990>

In [10]:
# set input feature class and old field name
input_fc = "City_Min"
old_field_name = "RASTERVALU"

# set output field name
new_field_name = "Min_Temp"

# check if new field name already exists
if new_field_name not in [f.name for f in arcpy.ListFields(input_fc)]:
    # add new field with same properties as old field
    field_obj = [f for f in arcpy.ListFields(input_fc) if f.name == old_field_name][0]
    arcpy.AddField_management(input_fc, new_field_name, field_obj.type, field_obj.precision, field_obj.scale, field_obj.length)

    # populate new field with values from old field
    with arcpy.da.UpdateCursor(input_fc, [old_field_name, new_field_name]) as cursor:
        for row in cursor:
            row[1] = row[0]
            cursor.updateRow(row)

    # delete old field
    arcpy.DeleteField_management(input_fc, old_field_name)

else:
    print("New field name already exists!")


In [11]:
arcpy.sa.ExtractValuesToPoints(
    in_point_features="City_Min.shp",
    in_raster="Kriging_max.tif",
    out_point_features=r"City_Temp",
    interpolate_values="NONE",
    add_attributes="VALUE_ONLY"
)

<geoprocessing server result object object at 0x00000297F4358630>

In [12]:
# set input feature class and old field name
input_fc = "City_Temp"
old_field_name = "RASTERVALU"

# set output field name
new_field_name = "Max_Temp"

# check if new field name already exists
if new_field_name not in [f.name for f in arcpy.ListFields(input_fc)]:
    # add new field with same properties as old field
    field_obj = [f for f in arcpy.ListFields(input_fc) if f.name == old_field_name][0]
    arcpy.AddField_management(input_fc, new_field_name, field_obj.type, field_obj.precision, field_obj.scale, field_obj.length)

    # populate new field with values from old field
    with arcpy.da.UpdateCursor(input_fc, [old_field_name, new_field_name]) as cursor:
        for row in cursor:
            row[1] = row[0]
            cursor.updateRow(row)

    # delete old field
    arcpy.DeleteField_management(input_fc, old_field_name)

else:
    print("New field name already exists!")